In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [32]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


def solution():
    
    #Read the file as a pandas data-frame
    df = pd.read_csv('/content/gdrive/MyDrive/01 - NLP Mayank Files/Eopinions.csv')
    #df = pd.read_csv('res/Eopinions.csv')
    
    ###################################################################################
    ##############   Perform Label Encoding on ‘class’ column. ########################
    ###################################################################################
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df['class'] = le.fit_transform(df['class'])
    print(df.head())
    
    ###########################################################################################
    ##############   Preprocess the text column        ########################################
    ##### Here we are definining only function. We will use it together with countvectorizer #
    ##########################################################################################
    from nltk.stem import WordNetLemmatizer
    from nltk.tokenize import RegexpTokenizer
    from nltk.corpus import stopwords
    
    # Below function is from Edureka Sample Code - 

    def preprocess_text(str):
      import re
      # Remove URLs from the text
      str = re.sub(r'^https?:\/\/.*[\r\n]*', '', str, flags=re.MULTILINE)
      
      # Remove digits
      str = re.sub("\d+", "", str)
      
      # Remove and replace "'", "--", "-","[","]" by " "
      str = re.sub(r'[\]\[\-\--\/.\'\,(;:)\\"!?]',r' ', str, flags=re.MULTILINE)
      
      # Remove non-alphabets
      str = re.sub('[^a-zA-Z^ ]', '', str, flags=re.MULTILINE)
      
      # Remove stop words and stemming
      ps = PorterStemmer()
      stopWords = set(stopwords.words('english'))
      words = word_tokenize(str)
      clean_text = ''
      for word in words:
        if word not in stopWords:
          # Stemming
          word = ps.stem(word)
          clean_text = clean_text+' '+word
      
      return clean_text
    

    #### Preprocess text using above function and store in original dataframe - 
    for i in range(len(df)):
      df.loc[i, 'text'] = preprocess_text(df.loc[i, 'text'].lower())
    
    print(df.head())

    ################################################################################################
    #Split the dataset into 2 parts namely “train.csv” and “test.csv” having 80% and 20% of the data
    ################################################################################################
    from sklearn.model_selection import train_test_split
    X = df['text']
    y = df['class']
    X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X,y,test_size=0.20,random_state = 42)
    #print(X_train_cv.shape, X_test_cv.shape, y_train_cv.shape, y_test_cv.shape)

    ############################################################################################################################# 
    ##############   Vectorize the text using CountVectorizer and implement RandomForectCLassifier ###############################
    #############################################################################################################################
    # Import module
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.pipeline import Pipeline

    # Vectorize the text using CountVectorizer
    model = Pipeline([('CountVectorizer', CountVectorizer(ngram_range=(1,2),max_df=0.3,min_df=7)),
                      ('Classifier Model', RandomForestClassifier())])
    
    model.fit(X_train_cv, y_train_cv)
    
    y_pred_rf = model.predict(X_test_cv)
    conf_mat_rf = confusion_matrix(y_test_cv, y_pred_rf)
    print('Confusion Matrix:\n',conf_mat_rf)

In [33]:
solution()

   class                                               text
0      0   I have recently purchased a J30T with moderat...
1      1   I bought this product because I need instant ...
2      0   I have owned my Buick since 53000 km and I am...
3      1   This was my first Digital camera so I did qui...
4      1   Minolta DiMAGE 7Hi is in a digital SLR with 5...
   class                                               text
0      0   recent purchas jt moder mile shop car look pa...
1      1   bought product need instant gratif stand tort...
2      0   own buick sinc km approach must say nicest ca...
3      1   first digit camera quit bit research unfortun...
4      1   minolta dimag hi digit slr megapixel ccd sens...
Confusion Matrix:
 [[46  0]
 [ 0 74]]
